<a href="https://colab.research.google.com/github/worldofaryavart/colab_notebooks/blob/colabnotebook/making_scraperModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers spacy wordnet nltk

  Preparing metadata (setup.py) ... done
  Created wheel for wordnet: filename=wordnet-0.0.1b2-py3-none-any.whl size=10498 sha256=510e1df90ebc6d15225240d1ae4b42dfac0f44e896a482be9e1e1c8c81047f49
  Stored in directory: /root/.cache/pip/wheels/c0/a1/e8/4649c8712033dcdbd1e64a0fc75216a5d1769665852c36b4f9
Successfully built wordnet


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import spacy
import nltk
from nltk.corpus import wordnet
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
class QueryDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length=128):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = self.labels[idx]

    encoding = self.tokenizer(
        text,
        add_special_tokens=True,
        max_length=self.max_length,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(label, dtype=torch.long)
    }

In [6]:
class IntentClassifier(nn.Module):
  def __init__(self, n_classes, pretrained_model="bert-base-uncased"):
    super().__init__()
    self.bert = AutoModel.from_pretrained(pretrained_model)
    self.drop = nn.Dropout(p=0.3)
    self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    output = self.drop(outputs[1])
    return self.fc(output)

In [7]:
class QueryExpander:
  def __init__(self):
    self.nlp = spacy.load('en_core_web_sm')

  def get_synonyms(self, word):
    synonyms = set()
    for syn in wordnet.synsets(word):
      for lemma in syn.lemmas():
        synonyms.add(lemma.name())
    return list(synonyms)

  def expand_query(self, query):
    doc = self.nlp(query)
    expanded_terms = []

    for token in doc:
      if token.pos_ in ['NOUN', 'VERB', 'ADJ']:
        synonyms = self.get_synonyms(token.text)
        expanded_terms.extend(synonyms[:2])

    return list(set([term.lower() for term in expanded_terms]))


In [8]:
def create_sample_dataset():
  """Create a sample dataset for intent classification"""
  queries = [
      "Find reasearch papers about quantum computing",
      "Download PDF papers on machine learning",
      "Summarize recent articles about AI",
      "Show me videos explaining neural networks",
      "Get images of black holes",
  ]

  intents = [
        "research_retrieval",
        "pdf_download",
        "summarization",
        "video_search",
        "image_search",
        # Add corresponding intents...
  ]

  return pd.DataFrame({'query': queries, 'intent': intents})


In [9]:
def train_intent_classifier(model, train_loader, device, epochs=3):
  optimizer = optim.Adam(model.parameters(), lr=2e-5)
  criterion = nn.CrossEntropyLoss()

  for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      optimizer.zero_grad()
      outputs = model(input_ids, attention_mask)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")

In [10]:
def main():
  df = create_sample_dataset()
  label_encoder = LabelEncoder()
  df['encoded_intent'] = label_encoder.fit_transform(df['intent'])

  train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  n_classes = len(label_encoder.classes_)
  model = IntentClassifier(n_classes)

  train_dataset = QueryDataset(
        texts=train_df['query'].values,
        labels=train_df['encoded_intent'].values,
        tokenizer=tokenizer
    )

  train_loader = DataLoader(
      train_dataset,
      batch_size=8,
      shuffle=True
  )

  query_expander = QueryExpander()

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  train_intent_classifier(model, train_loader, device)

  test_query = "Find recent papers about deep learning"

  expanded_terms = query_expander.expand_query(test_query)
  print(f"Expanded terms: {expanded_terms}")

  model.eval()
  with torch.no_grad():
    encoding = tokenizer(
        test_query,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    outputs = model(
        input_ids=encoding['input_ids'].to(device),
        attention_mask=encoding['attention_mask'].to(device)
    )
    predicted_intent = label_encoder.inverse_transform([outputs.argmax().item()])[0]
    print(f"Predicted intent: {predicted_intent}")

if __name__ == "__main__":
  main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1, Average Loss: 1.5012
Epoch 2, Average Loss: 1.7414
Epoch 3, Average Loss: 1.4054
Expanded terms: ['oceanic_abyss', 'recover', 'regain', 'thick', 'memorize', 'get_word', 'report', 'wallpaper', 'recent_epoch', 'holocene_epoch']
Predicted intent: image_search


In [2]:
!pip install playwright beautifulsoup4 pytesseract pillow PyMuPDF youtube_dl transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.0 MB/s eta 0:00:00


In [4]:
!playwright install chromium

164.5 MiB [] 0% 0.0s164.5 MiB [] 0% 81.6s164.5 MiB [] 0% 52.8s164.5 MiB [] 0% 22.8s164.5 MiB [] 0% 19.1s164.5 MiB [] 0% 18.0s164.5 MiB [] 0% 16.3s164.5 MiB [] 0% 13.6s164.5 MiB [] 1% 12.5s164.5 MiB [] 1% 13.0s164.5 MiB [] 1% 12.7s164.5 MiB [] 1% 11.6s164.5 MiB [] 1% 11.1s164.5 MiB [] 2% 11.0s164.5 MiB [] 2% 10.9s164.5 MiB [] 2% 10.8s164.5 MiB [] 2% 11.1s164.5 MiB [] 2% 10.8s164.5 MiB [] 2% 10.9s164.5 MiB [] 3% 10.5s164.5 MiB [] 3% 10.2s164.5 MiB [] 3% 10.5s164.5 MiB [] 4% 10.4s164.5 MiB [] 4% 10.3s164.5 MiB [] 4% 10.7s164.5 MiB [] 4% 10.9s164.5 MiB [] 4% 10.5s164.5 MiB [] 5% 10.1s164.5 MiB [] 5% 9.7s164.5 MiB [] 5% 9.6s164.5 MiB [] 6% 9.2s164.5 MiB [] 6% 8.7s164.5 MiB [] 6% 8.6s164.5 MiB [] 7% 8.5s164.5 MiB [] 7% 8.4s164.5 MiB [] 7% 8.2s164.5 MiB [] 8% 7.9s164.5 MiB [] 8% 7.8s164.5 MiB [] 8% 7.6s164.5 MiB [] 9% 7.5s164.5 MiB [] 9% 7.3s164.5 MiB [] 9% 7.4s164.5 MiB [] 10% 7.3s164.5 MiB [] 10% 7.2s164.5 MiB [] 10% 7.1s164.5 MiB [] 11% 7.0s164.5 MiB [] 11% 7.1s164.5 MiB [] 12% 7.1s164.5 M

In [5]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (377 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [6]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,038 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123653 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [12]:
import asyncio
from playwright.async_api import async_playwright
import aiohttp
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pytesseract
from PIL import Image
import io
import fitz  # PyMuPDF
import re
from transformers import pipeline
import threading
from queue import Queue
import time

In [13]:
!pip install nest_asyncio

In [14]:
import nest_asyncio

In [16]:
nest_asyncio.apply()

In [18]:
class SuperPoweredCrawler:
    def __init__(self):
        self.visited_urls = set()
        self.url_queue = Queue()
        self.results = []
        self.content_lock = threading.Lock()

        # Initialize content extractors
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cpu")

        # Configurable crawler settings
        self.max_depth = 3
        self.max_pages_per_domain = 1000
        self.concurrent_requests = 5  # Reduced for stability

        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
        }

    async def start_crawl(self, seed_urls, search_query):
        """
        Start the crawling process with multiple seed URLs
        """
        print(f"Starting crawl for query: {search_query}")

        for url in seed_urls:
            self.url_queue.put((url, 0))  # (url, depth)

        async with async_playwright() as playwright:
            browser = await playwright.chromium.launch(headless=True)
            context = await browser.new_context(
                viewport={'width': 1280, 'height': 800},
                user_agent=self.headers['User-Agent'],
                extra_http_headers=self.headers
            )

            # Start crawler workers
            tasks = []
            for _ in range(self.concurrent_requests):
                task = asyncio.create_task(self.crawler_worker(context, search_query))
                tasks.append(task)

            try:
              await asyncio.wait_for(asyncio.gather(*task), timeout=300)
            except asyncio.TimeoutError:
              print("Crawling timed out after 5 minutes")
            finally:
              await browser.close()

        return self.results

    async def process_page(self, page, content, url):
        """
        Process page content based on content type
        """
        try:
            # Get content type from response headers
            response = await page.goto(url)
            if not response:
                print(f"No response from {url}")
                return None

            content_type = response.headers.get('content-type', '').lower()

            if 'pdf' in content_type:
                return await self.process_pdf_content(response, url)
            elif any(img_type in content_type for img_type in ['image/jpeg', 'image/png', 'image/gif']):
                return await self.process_image_content(response, url)
            else:
                return await self.process_html_content(page, content, url)

        except Exception as e:
            print(f"Error processing page {url}: {str(e)}")
            return None

    async def crawler_worker(self, context, search_query):
        """
        Worker process for crawling pages
        """
        while not self.url_queue.empty():
            try:
                url, depth = self.url_queue.get_nowait()
            except:
                break

            if depth > self.max_depth or url in self.visited_urls:
                continue

            try:
                print(f"Crawling: {url}")
                page = await context.new_page()

                try:
                  await page.goto(
                      url,
                      wait_until='domcontentloaded',
                      timeout=30000
                  )
                except Exception as e:
                  print(f"Navigation error for {url}: {str(e)}")
                  await page.close()
                  continue

                # Get page content
                content = await page.content()

                # Process the page content
                result = await self.process_page(page, content, url)
                if result:
                    self.add_result(result, search_query)

                # Extract and queue new URLs
                new_urls = await self.extract_urls(page)
                for new_url in new_urls:
                    if self.should_crawl(new_url):
                        self.url_queue.put((new_url, depth + 1))

                self.visited_urls.add(url)
                await page.close()

                # Add a small delay between requests
                await asyncio.sleep(2)

            except Exception as e:
                print(f"Error crawling {url}: {str(e)}")
                if 'page' in locals():
                    await page.close()
                continue
    def should_crawl(self, url):
      """Determine if a URL should be crawled"""
      try:
        parsed = urlparse(url)

        if not parsed.scheme in ['http', 'https']:
          return False

        domain = parsed.netloc
        domain_count = sum(1 for visited in self.visited_urls
                           if urlparse(visited).netloc == domain)
        if domain_count >= self.max_pages_per_domain:
          return False

        exclude_patterns = [
                r'\.(css|js|json|xml)$',
                r'(login|signup|logout)',
                r'(facebook|twitter|instagram)',
                r'\.(jpg|jpeg|png|gif)$',  # Skip direct image links
                r'\/api\/',
                r'\/rss\/',
                r'\/feed\/',
                r'\/search\?',
                r'\/page\/\d+',
            ]

        return not any(re.search(pattern, url, re.I)
                         for pattern in exclude_patterns)

      except:
            return False

    async def process_html_content(self, page, content, url):
        """
        Process HTML content
        """
        try:
            soup = BeautifulSoup(content, 'html.parser')

            # Extract text content
            text_content = ' '.join([p.get_text() for p in soup.find_all(['p', 'article', 'section'])])
            text_content = re.sub(r'\s+', ' ', text_content).strip()

            # Get title
            title = await page.title()

            # Generate summary if content is long enough
            summary = None
            if len(text_content) > 500:
                try:
                    summary = self.summarizer(text_content[:4096], max_length=150, min_length=50)[0]['summary_text']
                except Exception as e:
                    print(f"Error generating summary: {str(e)}")

            return {
                'url': url,
                'type': 'html',
                'title': title,
                'content': text_content[:5000],  # Limit content length
                'summary': summary,
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing HTML content for {url}: {str(e)}")
            return None

    async def process_pdf_content(self, response, url):
        """
        Process PDF content
        """
        try:
            pdf_data = await response.body()
            pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
            text_content = ""

            for page_num in range(min(pdf_document.page_count, 10)):  # Limit to first 10 pages
                page = pdf_document[page_num]
                text_content += page.get_text()

            return {
                'url': url,
                'type': 'pdf',
                'content': text_content[:5000],  # Limit content length
                'page_count': pdf_document.page_count,
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing PDF {url}: {str(e)}")
            return None

    async def process_image_content(self, response, url):
        """
        Process image content
        """
        try:
            image_data = await response.body()
            image = Image.open(io.BytesIO(image_data))

            # Extract text using OCR
            try:
                ocr_text = pytesseract.image_to_string(image)
            except:
                ocr_text = ""

            return {
                'url': url,
                'type': 'image',
                'ocr_text': ocr_text,
                'metadata': {
                    'width': image.size[0],
                    'height': image.size[1],
                    'format': image.format
                },
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing image {url}: {str(e)}")
            return None

    async def extract_urls(self, page):
        """
        Extract URLs from the page
        """
        try:
            # Get all links using JavaScript evaluation
            links = await page.evaluate('''() => {
                const links = Array.from(document.getElementsByTagName('a'));
                return links.map(link => link.href).filter(href => href);
            }''')

            return list(set(links))  # Remove duplicates

        except Exception as e:
            print(f"Error extracting URLs: {str(e)}")
            return []

    def should_crawl(self, url):
        """
        Determine if a URL should be crawled
        """
        try:
            parsed = urlparse(url)

            # Basic URL filtering
            if not parsed.scheme in ['http', 'https']:
                return False

            # Check domain crawl limit
            domain = parsed.netloc
            domain_count = sum(1 for visited in self.visited_urls
                             if urlparse(visited).netloc == domain)

            if domain_count >= self.max_pages_per_domain:
                return False

            # Filter out non-content URLs
            exclude_patterns = [
                r'\.(css|js|json|xml)$',
                r'(login|signup|logout)',
                r'(facebook|twitter|instagram)',
                r'\.(jpg|jpeg|png|gif)$'  # Skip direct image links
            ]

            return not any(re.search(pattern, url, re.I)
                         for pattern in exclude_patterns)

        except:
            return False

    def add_result(self, result, search_query):
        """
        Add processed result to the results list with relevance scoring
        """
        with self.content_lock:
            result['relevance_score'] = self.calculate_relevance(result, search_query)
            self.results.append(result)
            self.results.sort(key=lambda x: x['relevance_score'], reverse=True)

    def calculate_relevance(self, result, query):
        """
        Calculate relevance score for a result
        """
        score = 0
        query_terms = query.lower().split()

        # Get the content to score
        content = ''
        if 'content' in result:
            content = result['content'].lower()
        elif 'ocr_text' in result:
            content = result['ocr_text'].lower()

        # Term frequency scoring
        for term in query_terms:
            score += content.count(term)

        # Type-based boosting
        type_boost = {
            'pdf': 1.2,
            'html': 1.0,
            'image': 0.8
        }
        score *= type_boost.get(result['type'], 1.0)

        return score

async def run_crawler():
    # Example usage
    crawler = SuperPoweredCrawler()

    # Define seed URLs
    seed_urls = [
    "https://research.ibm.com/quantum-computing",  # Updated IBM URL
    "https://www.nature.com/subjects/quantum-physics",  # Alternative Nature URL
    "https://www.scientificamerican.com/computing/",
    "https://www.quantum-computing.news/",  # Alternative quantum computing news
    "https://quantumcomputing.stackexchange.com/"  # More reliable source
    ]

    # Start the crawl
    results = await crawler.start_crawl(seed_urls, "quantum computing latest developments")

    # Print results
    print("\nSearch Results:")
    for i, result in enumerate(results[:10], 1):
        print(f"\n{i}. {result['url']}")
        print(f"Type: {result['type']}")
        print(f"Score: {result['relevance_score']:.2f}")
        if 'summary' in result and result['summary']:
            print(f"Summary: {result['summary']}")

await run_crawler()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Starting crawl for query: quantum computing latest developments
Crawling: https://research.ibm.com/quantum-computing
Crawling: https://www.nature.com/subjects/quantum-physics
Crawling: https://www.scientificamerican.com/computing/
Crawling: https://www.quantum-computing.news/
Crawling: https://quantumcomputing.stackexchange.com/
Error crawling https://research.ibm.com/quantum-computing: BrowserContext.new_page: Target page, context or browser has been closed
Error crawling https://www.nature.com/subjects/quantum-physics: BrowserContext.new_page: Target page, context or browser has been closed
Error crawling https://www.scientificamerican.com/computing/: BrowserContext.new_page: Target page, context or browser has been closed


RuntimeError: await wasn't used with future

Error crawling https://www.quantum-computing.news/: BrowserContext.new_page: Target page, context or browser has been closed
Error crawling https://quantumcomputing.stackexchange.com/: BrowserContext.new_page: Target page, context or browser has been closed


In [30]:
class SuperPoweredCrawler:
    def __init__(self):
        self.visited_urls = set()
        self.url_queue = Queue()
        self.results = []
        self.content_lock = threading.Lock()

        # Initialize content extractors
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

        # Configurable crawler settings
        self.max_depth = 3
        self.max_pages_per_domain = 1000
        self.respect_robots = False  # Note: Be careful with this setting
        self.concurrent_requests = 50

        # Content type handlers
        self.content_handlers = {
            'text/html': self.process_html,
            'application/pdf': self.process_pdf,
            'image': self.process_image,
            'video': self.process_video
        }

    async def start_crawl(self, seed_urls, search_query):
        """
        Start the crawling process with multiple seed URLs
        """
        print(f"Starting crawl for query: {search_query}")

        for url in seed_urls:
            self.url_queue.put((url, 0))  # (url, depth)

        async with async_playwright() as playwright:
            browser = await playwright.chromium.launch(headless=True)

            # Create multiple browser contexts for parallel processing
            contexts = [await browser.new_context() for _ in range(self.concurrent_requests)]

            # Start crawler workers
            tasks = []
            for context in contexts:
                task = asyncio.create_task(self.crawler_worker(context, search_query))
                tasks.append(task)

            await asyncio.gather(*tasks)
            await browser.close()

        return self.results

    async def crawler_worker(self, context, search_query):
        """
        Worker process for crawling pages
        """
        while not self.url_queue.empty():
            try:
                url, depth = self.url_queue.get_nowait()
            except:
                continue

            if depth > self.max_depth or url in self.visited_urls:
                continue

            try:
                page = await context.new_page()
                await page.goto(url, wait_until='networkidle', timeout=30000)

                # Get page content and metadata
                content = await page.content()
                title = await page.title()

                # Determine content type
                content_type = await self.detect_content_type(page)

                # Process content based on type
                if content_type in self.content_handlers:
                    result = await self.content_handlers[content_type](page, content, url)
                    if result:
                        self.add_result(result, search_query)

                # Extract new URLs
                new_urls = await self.extract_urls(page)
                for new_url in new_urls:
                    if self.should_crawl(new_url):
                        self.url_queue.put((new_url, depth + 1))

                await page.close()

            except Exception as e:
                print(f"Error crawling {url}: {str(e)}")
                continue

            self.visited_urls.add(url)

    async def detect_content_type(self, page):
        """
        Detect the type of content on the page
        """
        # Check response headers
        response = await page.main_frame.request.response()
        if response:
            content_type = response.headers.get('content-type', '')

            if 'pdf' in content_type:
                return 'application/pdf'
            elif any(img_type in content_type for img_type in ['image/jpeg', 'image/png', 'image/gif']):
                return 'image'
            elif 'video' in content_type:
                return 'video'

        return 'text/html'

    async def process_html(self, page, content, url):
        """
        Process HTML content and extract relevant information
        """
        soup = BeautifulSoup(content, 'html.parser')

        # Extract text content
        text_content = ' '.join([p.get_text() for p in soup.find_all('p')])

        # Extract metadata
        meta_tags = soup.find_all('meta')
        metadata = {
            tag.get('name', tag.get('property', '')): tag.get('content', '')
            for tag in meta_tags
        }

        # Get main content using readability algorithms
        main_content = await self.extract_main_content(page)

        # Generate summary if content is long enough
        summary = None
        if len(main_content) > 500:
            summary = self.summarizer(main_content, max_length=150, min_length=50)[0]['summary_text']

        return {
            'url': url,
            'type': 'html',
            'title': await page.title(),
            'content': main_content,
            'summary': summary,
            'metadata': metadata,
            'timestamp': time.time()
        }

    async def process_pdf(self, page, content, url):
        """
        Extract and process PDF content
        """
        try:
            # Download PDF content
            response = await page.main_frame.request.response()
            pdf_content = await response.body()

            # Process PDF using PyMuPDF
            pdf_document = fitz.open(stream=pdf_content, filetype="pdf")
            text_content = ""
            images = []

            for page_num in range(pdf_document.page_count):
                page = pdf_document[page_num]
                text_content += page.get_text()

                # Extract images
                image_list = page.get_images(full=True)
                for img_index, img in enumerate(image_list):
                    xref = img[0]
                    base_image = pdf_document.extract_image(xref)
                    image_bytes = base_image["image"]
                    images.append(image_bytes)

            return {
                'url': url,
                'type': 'pdf',
                'content': text_content,
                'images': images,
                'page_count': pdf_document.page_count,
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing PDF {url}: {str(e)}")
            return None

    async def process_image(self, page, content, url):
        """
        Process and analyze image content
        """
        try:
            # Get image data
            response = await page.main_frame.request.response()
            image_data = await response.body()

            # Convert to PIL Image
            image = Image.open(io.BytesIO(image_data))

            # Extract text using OCR
            ocr_text = pytesseract.image_to_string(image)

            # Basic image analysis
            width, height = image.size
            format = image.format
            mode = image.mode

            return {
                'url': url,
                'type': 'image',
                'ocr_text': ocr_text,
                'metadata': {
                    'width': width,
                    'height': height,
                    'format': format,
                    'mode': mode
                },
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing image {url}: {str(e)}")
            return None

    async def process_video(self, page, content, url):
        """
        Extract information from video content
        """
        try:
            # Video metadata extraction using youtube-dl
            ydl_opts = {
                'format': 'best',
                'extract_flat': True,
                'force_generic_extractor': True
            }

            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                video_info = ydl.extract_info(url, download=False)

            return {
                'url': url,
                'type': 'video',
                'title': video_info.get('title'),
                'description': video_info.get('description'),
                'duration': video_info.get('duration'),
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing video {url}: {str(e)}")
            return None

    def add_result(self, result, search_query):
        """
        Add processed result to the results list with relevance scoring
        """
        with self.content_lock:
            # Calculate relevance score
            relevance_score = self.calculate_relevance(result, search_query)
            result['relevance_score'] = relevance_score

            self.results.append(result)
            # Sort results by relevance
            self.results.sort(key=lambda x: x['relevance_score'], reverse=True)

    def calculate_relevance(self, result, query):
        """
        Calculate relevance score for a result
        """
        score = 0
        query_terms = query.lower().split()

        if 'content' in result:
            content = result['content'].lower()
            # Term frequency scoring
            for term in query_terms:
                score += content.count(term)

            # Position scoring
            for term in query_terms:
                pos = content.find(term)
                if pos != -1:
                    score += 1 / (pos + 1)

        # Type-based boosting
        type_boost = {
            'pdf': 1.2,  # Boost PDFs as they often contain detailed information
            'html': 1.0,
            'video': 1.1,
            'image': 0.9
        }
        score *= type_boost.get(result['type'], 1.0)

        return score

    async def extract_urls(self, page):
        """
        Extract URLs from the page
        """
        links = await page.evaluate('''() => {
            const links = Array.from(document.getElementsByTagName('a'));
            return links.map(link => link.href);
        }''')

        return [link for link in links if self.should_crawl(link)]

    def should_crawl(self, url):
        """
        Determine if a URL should be crawled
        """
        try:
            parsed = urlparse(url)

            # Basic URL filtering
            if not parsed.scheme in ['http', 'https']:
                return False

            # Check domain crawl limit
            domain = parsed.netloc
            domain_count = sum(1 for visited in self.visited_urls
                             if urlparse(visited).netloc == domain)

            if domain_count >= self.max_pages_per_domain:
                return False

            # Filter out common non-content URLs
            exclude_patterns = [
                r'\.(css|js|json|xml)$',
                r'(login|signup|logout)',
                r'(facebook|twitter|instagram)'
            ]

            return not any(re.search(pattern, url, re.I)
                         for pattern in exclude_patterns)

        except:
            return False

    async def extract_main_content(self, page):
        """
        Extract main content from webpage using readability
        """
        script = """
        () => {
            function getMainContent() {
                // Priority elements to check
                const priorities = ['article', 'main', '.content', '#content'];

                for (const selector of priorities) {
                    const element = document.querySelector(selector);
                    if (element) {
                        return element.innerText;
                    }
                }

                // Fallback to largest text block
                let largest = '';
                document.querySelectorAll('p').forEach(p => {
                    const text = p.innerText;
                    if (text.length > largest.length) {
                        largest = text;
                    }
                });

                return largest;
            }

            return getMainContent();
        }
        """

        content = await page.evaluate(script)
        return content

In [31]:
class SearchEngine:
  def __init__(self):
    self.crawler = SuperPoweredCrawler()

  async def search(self, query, seed_urls=None):
    """Perform a search using the crawler"""
    if seed_urls is None:
      #Default seed URLs for different content types

      seed_urls = [
        "https://arxiv.org",
        "https://scholar.google.com",
        "https://github.com",
        "https://medium.com",
        "https://youtube.com",
        # Add more seed URLs
      ]

    results = await self.crawler.start_crawl(seed_urls, query) # This line was incorrectly indented. Fixed by aligning with 'if' block.
    return self.rank_results(results, query)

  def rank_results(self, results, query):
    """ Rank and filter search results"""

    seen_urls = set()
    unique_results = []

    for result in results:
      url = result['url']
      if url not in seen_urls:
        seen_urls.add(url)
        unique_results.append(result)

    return unique_results

In [32]:
async def main():
    engine = SearchEngine()
    query = "quantum computing latest developments"
    results = await engine.search(query)

    for result in results[:10]:  # Top 10 results
        print(f"\nURL: {result['url']}")
        print(f"Type: {result['type']}")
        print(f"Score: {result['relevance_score']}")
        if 'summary' in result:
            print(f"Summary: {result['summary']}")

if __name__ == "__main__":
    await main()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Starting crawl for query: quantum computing latest developments
Error crawling https://arxiv.org: 'Frame' object has no attribute 'request'
Error crawling https://medium.com: 'Frame' object has no attribute 'request'
Error crawling https://scholar.google.com: 'Frame' object has no attribute 'request'
Error crawling https://github.com: 'Frame' object has no attribute 'request'
Error crawling https://youtube.com: 'Frame' object has no attribute 'request'
